In [3]:
!pip install torch==2.2 torchvision torchaudio lightning transformers datasets chess numpy pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 25.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 12.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 3.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 12.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
import torch
from nanoGPT.model import GPTConfig
from lightning_training import *

In [5]:
elite_database = pd.read_csv("../data/lichess_elite_database.csv", nrows=10**7, delimiter=";")#, usecols=["result", "white_elo", "black_elo", "piece_uci"])
# elite_database = elite_database.sample(n = 10**7, random_state=42)
elite_database = elite_database.dropna()
elite_database = elite_database[['result', 'white_elo', 'black_elo', 'piece_uci']]
elite_database

,result,white_elo,black_elo,piece_uci
0,1-0,2271,2428,Pd2d4 Ng8f6 Pc2c4 Pe7e6 Nb1c3 Bf8b4 Ng1f3 Ke8g...
1,1-0,2271,2428,Pd2d4 Ng8f6 Pc2c4 Pe7e6 Nb1c3 Bf8b4 Ng1f3 Ke8g...
2,1-0,2423,2438,Pd2d4 Ng8f6 Pc2c4 Pc7c6 Nb1c3 Pd7d5 Pe2e3 Pe7e...
3,1-0,2439,2276,Pe2e4 Pd7d5 Pe4d5 Ng8f6 Pc2c4 Pc7c6 Pd2d4 Pc6d...
4,1-0,2347,2430,Pd2d4 Pd7d5 Pc2c4 Pc7c6 Pc4d5 Pc6d5 Nb1c3 Nb8c...
...,...,...,...,...
9999995,0-1,2460,2642,Pd2d4 Pe7e5 Pd4e5 Nb8c6 Bc1f4 Pd7d6 Pe5d6 Qd8f...
9999996,1-0,2599,2549,Pe2e4 Pe7e5 Pd2d4 Pe5d4 Bf1c4 Ng8f6 Ng1f3 Nb8c...
9999997,1-0,2518,2313,Ng1f3 Pd7d6 Pd2d4 Ng8f6 Ph2h3 Pc7c6 Pc2c4 Ph7h...
9999998,1-0,2777,2687,Pe2e4 Pc7c6 Pc2c4 Pd7d5 Pe4d5 Pc6d5 Pd2d4 Ng8f...


In [6]:
# data_module = WeightedGamesDataModule(elite_database, test_size=0.01, num_workers=16)
# games = list(elite_database.piece_uci)
data_module = WeightedGamesDataModule(elite_database, batch_size=64, test_size=0.01, num_workers=8)

In [7]:
model_config = GPTConfig(
    block_size=301,
    vocab_size=len(data_module.tokenizer.vocab),
    n_layer=4,
    n_head=4,
    n_embd=256,
    bias=False,
)

pl_model = LightningGPTWeighted(model_config)
# pl_model = LightningGPT(model_config)

number of parameters: 4.19M


In [8]:
ckpt_path = "./lightning_logs/strong_play/W1-D0.5-L0/version_1/checkpoints/epoch=0-step=154565.ckpt"

pl_model = LightningGPTWeighted.load_from_checkpoint(ckpt_path, config=model_config)

number of parameters: 4.19M


In [9]:
tensorboard_logger = pl.loggers.TensorBoardLogger(save_dir="./lightning_logs/strong_play/", name=f"W1-D0.5-L0")

trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=10,
    # callbacks=[pl.callbacks.RichProgressBar()],
    logger=tensorboard_logger,
    precision="bf16-mixed",
    # default_root_dir=
    # fast_dev_run=True
)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
torch.set_float32_matmul_precision("high")

In [ ]:
trainer.fit(
    model=pl_model,
    datamodule=data_module,
    ckpt_path=ckpt_path
)

Restoring states from the checkpoint path at ./lightning_logs/strong_play/W1-D0.5-L0/version_1/checkpoints/epoch=0-step=154565.ckpt
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:361: The dirpath has changed from './lightning_logs/strong_play/W1-D0.5-L0/version_1/checkpoints' to './lightning_logs/strong_play/W1-D0.5-L0/version_2/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | model         | GPT                | 4.3 M 
1 | test_accuracy | MulticlassAccuracy | 0     
-----------------------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.085    Total estimated model params size (MB)
Restored all states from t

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve

In [13]:
ckpt_path = "./lightning_logs/strong_play/W1-D0.5-L0/version_1/checkpoints/epoch=0-step=154565.ckpt"

In [12]:
!ls

__pycache__		     elo_bins_training.ipynb  training.ipynb
convert_pgn_to_csv.py	     lightning_logs	      weighted_training.ipynb
create_elo_bins_datasets.py  lightning_training.py
data_process		     nanoGPT


In [14]:
pl_model_2 = LightningGPTWeighted.load_from_checkpoint(ckpt_path, config=model_config)

number of parameters: 4.19M


In [15]:
pl_model_2.state_dict()

OrderedDict([('model.transformer.wte.weight',
              tensor([[ 0.0870, -0.4989,  0.0895,  ..., -0.2696, -0.1357,  0.4120],
                      [ 0.8548, -0.1887,  0.0328,  ..., -0.2452, -0.0648,  0.0368],
                      [ 0.0960, -0.4917,  0.0858,  ..., -0.2670, -0.1410,  0.4018],
                      ...,
                      [ 0.1217, -0.3253,  0.0677,  ..., -0.2096, -0.1154, -0.1193],
                      [ 0.2769, -0.6054,  0.1701,  ..., -0.1642, -0.0624, -0.1738],
                      [ 0.5010, -0.1428,  0.1870,  ..., -0.5709, -0.3208, -0.1402]],
                     device='cuda:0')),
             ('model.transformer.wpe.weight',
              tensor([[ 8.7992e-01, -1.3818e-01,  4.3597e-02,  ..., -2.7330e-01,
                       -4.6575e-02,  2.2585e-02],
                      [ 2.7124e-02, -2.7916e-01,  1.5233e-01,  ...,  4.6013e-01,
                       -1.7155e-01,  5.4761e-01],
                      [ 9.8444e-02,  4.7243e-02, -1.2057e-01,  ..., -4.639